In [2]:
import pymc3 as pm
import numpy as np
import scipy.stats as st
import theano.tensor as tt

In [ ]:
with pm.Model() as model1:
    step_size = pm.HalfNormal('ste-Size', sd=0.1)
    nu = pm.HalfNormal('nu', sd=10.)
    
    sigma = pm.GaussianRandomWalk('sigma', sd=step_size, shape=interval)
    sigma_weights = tt.repeat(sigma, len(time_series) // interval)
    
    y_ = pm.StudentT('y', mu=0.0, nu=nu, sd=pm.math.exp(sigma_weights), 
                     observed=time_series)
    
    trace = pm.sample(2000, tune=2000)
    

In [4]:
pm.GaussianRandomWalk.dist(0.024, shape=8).random(size=100).shape

(100, 8)

In [5]:
pm.GaussianRandomWalk.dist(0.024, shape=(8,3)).random(size=100).shape

(100, 8, 3)

### GaussianRandomWalk variable

In [8]:
bound=pm.distributions.dist_math.bound
alpha, beta = 1.5, 2.
observed = st.weibull_min(alpha, scale=beta).rvs(30)
t = len(observed)
observed

array([3.39730564, 1.38965055, 1.2298051 , 2.37122152, 0.97857576,
       1.89470696, 2.50884531, 3.01916275, 2.28974749, 1.6569365 ,
       1.06319826, 1.27101804, 1.79828298, 2.52963131, 2.35751028,
       1.81264662, 1.21719678, 1.81465747, 0.17625529, 2.05179476,
       2.92053825, 3.51066554, 1.61497934, 1.76152038, 1.60745834,
       2.82693174, 0.87735854, 0.36583424, 3.27295593, 1.36040676])

In [9]:
def Weibull_logp(alpha, beta, value):
    return bound(tt.log(alpha) - tt.log(beta)
                  + (alpha - 1) * tt.log(value / beta)
                  - (value / beta)**alpha,
                     value >= 0, alpha > 0, beta > 0)

In [6]:
with pm.Model():
    alpha = pm.InverseGamma('alpha',alpha = 1.0,beta = 1.0)
    sd    = pm.InverseGamma('sd',alpha = 2.0,beta = 2.0)
    s     = pm.GaussianRandomWalk('s',sd = sd,shape = t)
    x     = pm.DensityDist('x', Weibull_logp, observed=dict(alpha=alpha, beta=sd, value=observed))
    trace = pm.sample()

<ipython-input-6-0f8ebd0af3b2>:19: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample()
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [s, sd, alpha]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 79 seconds.


MissingInputError: Input 0 of the graph (indices start from 0), used to compute Elemwise{exp,no_inplace}(alpha_log__), was not provided and not given a value. Use the Theano flag exception_verbosity='high', for more information on this error.

In [14]:
bound=pm.distributions.dist_math.bound
alpha, beta = 1.5, 2.
observed = st.weibull_min(alpha, scale=beta).rvs(30)
t = len(observed)
with pm.Model() as random_walk_model:
    alpha = pm.InverseGamma('alpha', alpha = 1.0, beta = 1.0)
    sd    = pm.InverseGamma('sd', alpha = 2.0, beta = 2.0)
    s     = pm.GaussianRandomWalk('s',sd = sd, shape = t)
    x     = pm.Weibull('x',alpha = alpha, beta = s, observed = observed)
    trace = pm.sample()

SamplingError: Initial evaluation of model at starting point failed!
Starting values:
{'alpha_log__': array(-0.69314718), 'sd_log__': array(-0.40546511), 's': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

Initial evaluation results:
alpha_log__    -1.31
sd_log__       -0.80
s             -14.89
x               -inf
Name: Log-probability of test_point, dtype: float64